In [119]:
import pandas as pd
import random
from snowflake.connector.pandas_tools import pd_writer
from sqlalchemy import create_engine
from datetime import timedelta

In [238]:
items_df = pd.read_csv("data/season-1819.csv")

In [239]:
items_df["MatchName"] = items_df["HomeTeam"] + " - " + items_df["AwayTeam"]

In [240]:
items_df.index = items_df.index.rename("MatchId")
items_df = items_df.reset_index()

In [241]:
items_df["Hour"] = random.choices(range(15, 22), k=len(items_df))
items_df["Datetime"] = pd.to_datetime(items_df["Date"], format='%d/%m/%Y').astype(str) + " " + items_df["Hour"].astype(str) +":00:00"

items_df["Datetime"] = pd.to_datetime(items_df["Datetime"])

In [242]:
SELECTED_FEATURES: list[str] = ['MATCHID',
 'DIV',
 'DATETIME',
 'MATCHNAME',
 'FTHG',
 'FTAG',
 'FTR',
 'B365H',
 'B365A',
 'B365D']

In [243]:
items_id = items_df["MatchId"].unique().tolist()
users_id = random.sample(range(0, 1_000_000), 100_000)

In [244]:
interactions = list(zip([random.choice(users_id)for i in range(1_000_000)], [random.choice(items_id) for i in range(1_000_000)]))

In [245]:
interactions_df = pd.DataFrame(interactions, columns=["UserId", "MatchId"])

In [246]:
interactions_df = interactions_df.merge(items_df[["MatchId", "Datetime"]], on="MatchId").rename(columns={"Datetime": "MaxDatetime"})

In [247]:
interactions_df["Datetime"] = interactions_df["MaxDatetime"] - pd.to_timedelta(random.choices(range(6*24*60*60), k=len(interactions_df)), 's')

In [248]:
interactions_df["BetAmount"] = random.choices(range(1, 1000), k=len(interactions_df))

In [249]:
interactions_df["Selection"] = random.choices(["Home", "Away", "Draw"], k=len(interactions_df))

In [250]:
interactions_df.drop(columns="MaxDatetime")

,UserId,MatchId,Datetime,BetAmount,Selection
0,750373,65,2018-09-26 08:50:09,362,Away
1,471957,65,2018-09-23 15:11:58,385,Home
2,332842,65,2018-09-26 14:52:21,695,Home
3,508195,65,2018-09-22 06:17:37,994,Home
4,995122,65,2018-09-25 13:52:49,599,Home
...,...,...,...,...,...
999995,675271,334,2019-04-25 18:58:56,3,Draw
999996,43729,334,2019-04-24 09:16:31,179,Away
999997,276450,334,2019-04-26 21:18:35,670,Away
999998,445334,334,2019-04-27 02:32:07,578,Away


In [254]:
items_df.columns = items_df.columns.str.upper()
interactions_df.columns = interactions_df.columns.str.upper()

items_df["DATETIME"] = items_df["DATETIME"].dt.tz_localize('UTC')
interactions_df["DATETIME"] = interactions_df["DATETIME"].dt.tz_localize('UTC')

In [260]:
account_identifier = 'betclic.eu-west-1'
user = 'psteffen'
database_name = 'DATASCIENCE_DEV'
schema_name = 'INOVAXION'
authenticator = "externalbrowser"
warehouse = 'WH_SCIENTIST'
role = "R_DS_DEV_WRITE"

conn_string = f"snowflake://{user}@{account_identifier}/{database_name}/{schema_name}?warehouse={warehouse}&role={role}"
engine = create_engine(conn_string, connect_args={'authenticator': 'externalbrowser'})

#Create your DataFrame

table_name = 'items'

#What to do if the table exists? replace, append, or fail?

if_exists = 'replace'

#Write the data to Snowflake, using pd_writer to speed up loading

with engine.connect() as con:
        items_df[SELECTED_FEATURES].to_sql(name='items', con=con, if_exists=if_exists, method=pd_writer, index=False)
        interactions_df.drop(columns="MAXDATETIME").to_sql(name='interactions', con=con, if_exists=if_exists, method=pd_writer, index=False)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://betclic.okta.com/app/snowflake/exk1ujdentUJZXPEK2p7/sso/saml?SAMLRequest=lZLdctowEIVfxaNe25KgNEEDZAiQKS1p3WA6LXdCXoiwLbn6waRPX2FCJ71IZnqnWZ0jfbtnBzfHqowOYKzUaohoQlAESuhcqt0QrbK7%2BBpF1nGV81IrGKInsOhmNLC8Kms29u5RPcAvD9ZF4SFlWXsxRN4oprmVlilegWVOsOX4fsE6CWG10U4LXaIXlrcd3FowLhBeLLmVAe%2FRuZph3DRN0nQTbXa4QwjBpI%2BD6iR5d9EfQ0%2Bv6Ckm70%2F6oAjy9JntVqrzCN7C2pxFln3MsjROvy4zFI0vqBOtrK%2FALMEcpIDVw%2BIMYAPB7SybLOaTBHzchNHFNLFKN9uSFyB0VXsXXk3CCW8hx6XeydD4fDpEdSFzUZDNdfcgZ2TFBfT3Wixkmh7oZl3A756ffvOT8X6fFYveT4Gi75dkO6dk59Z6mKtTni6USKcbUxJTmtEPjPRZt59c967WKJoGKKm4a50X6A04UUqR6MLxFo7XNf7LjeFYUL%2FPQbnVp%2FWPdPa5U19hazU%2BxYvOG8NaADP63zkM8Ev38%2FJ9CXnMp6kOTE%2FRnTYVd6%2FHRRPaVmQeb1spg4rLcpznBqwNsZWlbiYGuAs77owHhEfnX%2F%2Fd8tEf&RelayState=65395 to aut

In [259]:
items_df[SELECTED_FEATURES]

,MATCHID,DIV,DATETIME,MATCHNAME,FTHG,FTAG,FTR,B365H,B365A,B365D
0,0,F1,2018-08-10 16:00:00+00:00,Marseille - Toulouse,4,0,H,1.40,8.00,4.50
1,1,F1,2018-08-11 21:00:00+00:00,Angers - Nimes,3,4,A,2.30,3.30,3.20
2,2,F1,2018-08-11 18:00:00+00:00,Lille - Rennes,3,1,H,2.50,3.10,3.10
3,3,F1,2018-08-11 18:00:00+00:00,Montpellier - Dijon,1,2,A,1.72,5.50,3.50
4,4,F1,2018-08-11 17:00:00+00:00,Nantes - Monaco,1,3,A,3.20,2.30,3.25
...,...,...,...,...,...,...,...,...,...,...
375,375,F1,2019-05-24 18:00:00+00:00,Nantes - Strasbourg,0,1,A,1.65,5.50,3.80
376,376,F1,2019-05-24 15:00:00+00:00,Nice - Monaco,2,0,H,3.10,2.25,3.50
377,377,F1,2019-05-24 15:00:00+00:00,Nimes - Lyon,2,3,A,3.40,2.00,3.80
378,378,F1,2019-05-24 15:00:00+00:00,Reims - Paris SG,3,1,H,6.50,1.45,5.00


In [257]:
interactions_df["DATETIME"]

0        2018-09-26 08:50:09+00:00
1        2018-09-23 15:11:58+00:00
2        2018-09-26 14:52:21+00:00
3        2018-09-22 06:17:37+00:00
4        2018-09-25 13:52:49+00:00
                    ...           
999995   2019-04-25 18:58:56+00:00
999996   2019-04-24 09:16:31+00:00
999997   2019-04-26 21:18:35+00:00
999998   2019-04-27 02:32:07+00:00
999999   2019-04-26 16:53:13+00:00
Name: DATETIME, Length: 1000000, dtype: datetime64[ns, UTC]

In [258]:
items_df["DATETIME"]

0     2018-08-10 16:00:00+00:00
1     2018-08-11 21:00:00+00:00
2     2018-08-11 18:00:00+00:00
3     2018-08-11 18:00:00+00:00
4     2018-08-11 17:00:00+00:00
                 ...           
375   2019-05-24 18:00:00+00:00
376   2019-05-24 15:00:00+00:00
377   2019-05-24 15:00:00+00:00
378   2019-05-24 15:00:00+00:00
379   2019-05-24 19:00:00+00:00
Name: DATETIME, Length: 380, dtype: datetime64[ns, UTC]

In [263]:
items_df[SELECTED_FEATURES].to_csv("items.csv")
interactions_df.drop(columns="MAXDATETIME").to_csv("interactions.csv")

In [176]:
items_df[SELECTED_FEATURES].columns.str.lower()

Index(['matchid', 'div', 'datetime', 'matchname', 'fthg', 'ftag', 'ftr',
       'b365h', 'b365a', 'b365d'],
      dtype='object')